# Twitter Scraping & NLP

In [1]:
import requests 
import json
from config import consumer_key, consumer_secret, access_key, access_secret, bearer_token

In [2]:
from textblob import TextBlob
import pandas as pd
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import time
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install gensim
import gensim
from gensim.parsing.preprocessing import remove_stopwords 
import torch
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer

# Display max column width 
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/memme11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Twitter API (Tweepy)

In [4]:
# Initialize and gain access to Twitter API
def initialize():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

api = initialize()

In [5]:
# Use get statement to see json format
url = 'https://api.twitter.com/1.1/search/tweets.json'
params = {'q': 'bitcoin',
          'tweet_mode': 'extended', 
         'lang': 'en', 
         'count': 2}
headers = {'authorization': 'Bearer '+bearer_token}

sample_data = requests.get(url, params=params, headers=headers).json()

sample_data

{'statuses': [{'created_at': 'Mon Jul 26 22:25:09 +0000 2021',
   'id': 1419785893125115908,
   'id_str': '1419785893125115908',
   'full_text': 'RT @flurbnb: $120 to one person in 34 hours\n\nRetweet &amp; follow @KNIFESURFER &amp; turn on his notifications 🔔\n\n#Giveaway #Giveaways #GiveawayAle…',
   'truncated': False,
   'display_text_range': [0, 148],
   'entities': {'hashtags': [{'text': 'Giveaway', 'indices': [114, 123]},
     {'text': 'Giveaways', 'indices': [124, 134]}],
    'symbols': [],
    'user_mentions': [{'screen_name': 'flurbnb',
      'name': 'Flur 🤫 | 📌 daily giveaways 📌 $FLUR #flurarmy',
      'id': 1184294344984952833,
      'id_str': '1184294344984952833',
      'indices': [3, 11]},
     {'screen_name': 'KNIFESURFER',
      'name': 'KNIFE_SURFER✨',
      'id': 1328496543998111745,
      'id_str': '1328496543998111745',
      'indices': [66, 78]}],
    'urls': []},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://tw

In [6]:
# Call on tweepy API and create dataframe
search_words = ("bitcoin", "etherium", "cardano")
crypto_data = pd.DataFrame()

def get_data(data):
    data = {
        'text': data.full_text,
        'date': data.created_at,
        'followers': data.user.followers_count,
        'favourites': data.user.favourites_count,
        'retweets': data.retweet_count
    }
    return data

for tweets in search_words:
    comp_tweets = api.search(q=tweets, lang = 'en', result_type = 'recent', count=250, tweet_mode='extended')
    
    for tweet in comp_tweets:
        row = get_data(tweet)
        crypto_data = crypto_data.append(row, ignore_index=True)
        
crypto_data

,date,favourites,followers,retweets,text
0,2021-07-26 22:25:14,1926.0,44.0,2335.0,"RT @ashwsbreal: $100 in #Bitcoin to 1 person , just retweet and be following me."
1,2021-07-26 22:25:14,4776.0,53.0,17.0,RT @TheRealLuisSan: #BabyDoge looking beautiful I hope #bitcoin keeps moving up 👀🚀
2,2021-07-26 22:25:13,51981.0,335.0,15.0,RT @francispouliot_: Don't worship Bitcoin Core developers. They are replaceable. I'm grateful for their work but it's often overrated. Wor…
3,2021-07-26 22:25:13,3487.0,19.0,0.0,"@pjmeersman @Stacks Case in point, let's see what Dorsey does. Will they choose to build on Stacks or create a new platform for Bitcoin defi. The price of Stacks is mostly irrelevant in this decision, me thinks."
4,2021-07-26 22:25:12,2855.0,1713.0,1.0,RT @Goodnessy2: There's often a large difference between what you want and what you think you want. \n#investing #btc #Bitcoin #invest #fore…
...,...,...,...,...,...
295,2021-07-26 22:17:15,10584.0,591.0,0.0,We've know this for year's 🤔 so we must be Senior Portfolio Strategists 🤓\n\n#ADA #Cardano #CryptoNews \nhttps://t.co/s2zn8QH5LP
296,2021-07-26 22:17:12,31408.0,590.0,3.0,RT @danny_cryptofay: Trivia: Did you know you can delegate your voting power on #ProjectCatalyst even today? It's super basic but it kinda…
297,2021-07-26 22:17:03,8638.0,114.0,28.0,RT @1CrypticPoet: 💥 World Economic Forum Outlines Speed of XRP\n\nhttps://t.co/gEATg07Sj4 #XRP #XRPCommunity #crypto #blockchain
298,2021-07-26 22:16:34,4590.0,69.0,0.0,"@optictopic @JoyRaptor @HermanoLejanoCA @IOHK_Charles And I'm not even saying ""buy Tezos"", don't buy Tezos it could also be overvalued and yet Cardano is valued 40x higher and is not as good as Tezos is!"


## Data Preprocessing

In [7]:
# Formatting
# Keep only tweets with over 1000 favourites
crypto_data = crypto_data.loc[crypto_data['favourites']>1000]

# Clean text column using Regex
crypto_data['cleaned_text'] = crypto_data['text']
clean_text = '(RT) @[\w]*:|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)'
crypto_data['cleaned_text'] = crypto_data['cleaned_text'].str.replace(clean_text, " ", regex=True)
crypto_data['cleaned_text'] = crypto_data['cleaned_text'].str.lower()

# Convert date dtype to datetime, set index, sort index and drop duplicates
crypto_data['date'] = pd.to_datetime(crypto_data['date'])
crypto_data = crypto_data.set_index('date').sort_index(ascending=False)
crypto_data.drop_duplicates(inplace=True)

crypto_data.head(30)

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,favourites,followers,retweets,text,cleaned_text
date,,,,,
2021-07-26 22:25:14,1926.0,44.0,2335.0,"RT @ashwsbreal: $100 in #Bitcoin to 1 person , just retweet and be following me.",100 in bitcoin to 1 person just retweet and be following me
2021-07-26 22:25:14,4776.0,53.0,17.0,RT @TheRealLuisSan: #BabyDoge looking beautiful I hope #bitcoin keeps moving up 👀🚀,babydoge looking beautiful i hope bitcoin keeps moving up
2021-07-26 22:25:13,51981.0,335.0,15.0,RT @francispouliot_: Don't worship Bitcoin Core developers. They are replaceable. I'm grateful for their work but it's often overrated. Wor…,don t worship bitcoin core developers they are replaceable i m grateful for their work but it s often overrated wor
2021-07-26 22:25:13,3487.0,19.0,0.0,"@pjmeersman @Stacks Case in point, let's see what Dorsey does. Will they choose to build on Stacks or create a new platform for Bitcoin defi. The price of Stacks is mostly irrelevant in this decision, me thinks.",case in point let s see what dorsey does will they choose to build on stacks or create a new platform for bitcoin defi the price of stacks is mostly irrelevant in this decision me thinks
2021-07-26 22:25:12,2855.0,1713.0,1.0,RT @Goodnessy2: There's often a large difference between what you want and what you think you want. \n#investing #btc #Bitcoin #invest #fore…,there s often a large difference between what you want and what you think you want investing btc bitcoin invest fore
2021-07-26 22:25:11,2533.0,66.0,0.0,#bitcoin\n\nAsia starting to wake up and ape into #BabyLink @BSCBabyLink \n\n#bscgem #bsc #shiba #everrise #babycake #babybake #doge #bnb,bitcoin asia starting to wake up and ape into babylink bscgem bsc shiba everrise babycake babybake doge bnb
2021-07-26 22:25:11,1081.0,11040.0,39.0,RT @0xEarthDefi: @tourws Check our Earth Defi!\nHosted on the State-of-the-art #PolygonNetwork \n#yieldfarming and swap router\n\n👉👉 https://t.…,check our earth defi hosted on the state of the art polygonnetwork yieldfarming and swap router
2021-07-26 22:25:11,1525.0,30.0,0.0,"Hey #Hexicans whats your rankings on the #PulseChain Leaderboard? I got 17,151 lol. Wednesday I might be able to throw more at it and break 10,000 position. #HEX $HEX $PLS #PLS #Bitcoin #ETH",hey hexicans whats your rankings on the pulsechain leaderboard i got 17 151 lol wednesday i might be able to throw more at it and break 10 000 position hex hex pls pls bitcoin eth
2021-07-26 22:25:11,11914.0,290.0,0.0,@yasuyuki3892 @Blockchain_Jay Lol bitcoin is now a pump and dump? Okay lol,jay lol bitcoin is now a pump and dump okay lol


## Tokenization

In [8]:
# Tokenizing Functions

def get_wordnet_pos(word):
# Map POS tag to the first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Function for tokenizing tweets (already cleaned using regex)
def second_clean(tweet):
    tweet = remove_stopwords(tweet) # remove stopwords with Gensim

    lemmatizer = WordNetLemmatizer()
    tokenized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(tweet)]
    
    # remove left over stop words with nltk
    tokenized = [token for token in tokenized if token not in stopwords.words("english")] 

    # remove non-alpha characters and keep the words of length >2 only
    tokenized = [token for token in tokenized if token.isalpha() and len(token)>2]

    return tokenized

# Function for joining tokenized list into string
def combine_tokens(tokenized): 
    non_tokenized = ' '.join([w for w in tokenized])
    return non_tokenized

In [9]:
# Execute function 
crypto_data['tokens'] = crypto_data['cleaned_text'].apply(lambda x: second_clean(x))
crypto_data['final_clean'] = crypto_data['tokens'].apply(lambda x: combine_tokens(x))

crypto_data

,favourites,followers,retweets,text,cleaned_text,tokens,final_clean
date,,,,,,,
2021-07-26 22:25:14,1926.0,44.0,2335.0,"RT @ashwsbreal: $100 in #Bitcoin to 1 person , just retweet and be following me.",100 in bitcoin to 1 person just retweet and be following me,"[bitcoin, person, retweet, follow]",bitcoin person retweet follow
2021-07-26 22:25:14,4776.0,53.0,17.0,RT @TheRealLuisSan: #BabyDoge looking beautiful I hope #bitcoin keeps moving up 👀🚀,babydoge looking beautiful i hope bitcoin keeps moving up,"[babydoge, look, beautiful, hope, bitcoin, keep, move]",babydoge look beautiful hope bitcoin keep move
2021-07-26 22:25:13,51981.0,335.0,15.0,RT @francispouliot_: Don't worship Bitcoin Core developers. They are replaceable. I'm grateful for their work but it's often overrated. Wor…,don t worship bitcoin core developers they are replaceable i m grateful for their work but it s often overrated wor,"[worship, bitcoin, core, developer, replaceable, grateful, work, overrate, wor]",worship bitcoin core developer replaceable grateful work overrate wor
2021-07-26 22:25:13,3487.0,19.0,0.0,"@pjmeersman @Stacks Case in point, let's see what Dorsey does. Will they choose to build on Stacks or create a new platform for Bitcoin defi. The price of Stacks is mostly irrelevant in this decision, me thinks.",case in point let s see what dorsey does will they choose to build on stacks or create a new platform for bitcoin defi the price of stacks is mostly irrelevant in this decision me thinks,"[case, point, let, dorsey, choose, build, stack, create, new, platform, bitcoin, defi, price, stack, irrelevant, decision, think]",case point let dorsey choose build stack create new platform bitcoin defi price stack irrelevant decision think
2021-07-26 22:25:12,2855.0,1713.0,1.0,RT @Goodnessy2: There's often a large difference between what you want and what you think you want. \n#investing #btc #Bitcoin #invest #fore…,there s often a large difference between what you want and what you think you want investing btc bitcoin invest fore,"[large, difference, want, think, want, invest, btc, bitcoin, invest, fore]",large difference want think want invest btc bitcoin invest fore
...,...,...,...,...,...,...,...
2021-07-26 21:28:55,9856.0,74.0,6.0,"RT @Nathanj518: Wait until the hodlers and crypto people find out about this one... NFT, Gaming, AidiSwap, token burn and redistribution to…",wait until the hodlers and crypto people find out about this one nft gaming aidiswap token burn and redistribution to,"[wait, hodlers, crypto, people, nft, game, aidiswap, token, burn, redistribution]",wait hodlers crypto people nft game aidiswap token burn redistribution
2021-07-26 21:27:54,1948.0,98.0,45.0,"RT @Newairdrop7: 🚀 Door #Airdrop\n🎁 GET 2 $DOOR (for first 10,000 users)\n\n➡️ Claim here : \n\n🔗https://t.co/k4HbjYXSVD\n\n- Enter ETH Address\n-…",door airdrop get 2 door for first 10 000 users claim here enter eth address,"[door, airdrop, door, user, claim, enter, eth, address]",door airdrop door user claim enter eth address
2021-07-26 21:27:22,8496.0,198.0,325.0,RT @MattWallace888: This has been a FANTASTIC week for #Bitcoin but #Dogecoin &amp; #Etherium have each gone up by more than Bitcoin &amp; any othe…,this has been a fantastic week for bitcoin but dogecoin amp etherium have each gone up by more than bitcoin amp any othe,"[fantastic, week, bitcoin, dogecoin, amp, etherium, bitcoin, amp, othe]",fantastic week bitcoin dogecoin amp etherium bitcoin amp othe


## NLP - Vader Sentiment Model

In [10]:
# Sentiment labels function 
def sentiment_labels(df, feature, value): 
    df.loc[df[value] > 0,feature] = 'positive'
    df.loc[df[value] == 0,feature] = 'neutral'
    df.loc[df[value] < 0,feature] = 'negative'

In [11]:
# Vader sentiment analysis
# define function and variable for SentimentIntensityAnalyzer()

sia = SentimentIntensityAnalyzer()


def vader_sentiment(df):
    
    target_col='final_clean'
    prefix = 'vader_clean_'
        
    scores_col=prefix+'scores'
    
    compound_col = prefix+'polarity'
    
    sentiment = prefix+'sentiment'
    
    df[scores_col] = df[target_col].apply(lambda x:sia.polarity_scores(x))
    
    df[compound_col] = df[scores_col].apply(lambda d: d['compound'])
    
    sentiment_labels(df, sentiment, compound_col)

In [12]:
#Execute vader function
start = time.time()

vader_sentiment(crypto_data)
stop = time.time()

print(f'Vader analysis took: {round((stop-start)/60, 3)}minutes')

crypto_data.head(30)

Vader analysis took: 0.001minutes


,favourites,followers,retweets,text,cleaned_text,tokens,final_clean,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment
date,,,,,,,,,,
2021-07-26 22:25:14,1926.0,44.0,2335.0,"RT @ashwsbreal: $100 in #Bitcoin to 1 person , just retweet and be following me.",100 in bitcoin to 1 person just retweet and be following me,"[bitcoin, person, retweet, follow]",bitcoin person retweet follow,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-26 22:25:14,4776.0,53.0,17.0,RT @TheRealLuisSan: #BabyDoge looking beautiful I hope #bitcoin keeps moving up 👀🚀,babydoge looking beautiful i hope bitcoin keeps moving up,"[babydoge, look, beautiful, hope, bitcoin, keep, move]",babydoge look beautiful hope bitcoin keep move,"{'neg': 0.0, 'neu': 0.424, 'pos': 0.576, 'compound': 0.7783}",0.7783,positive
2021-07-26 22:25:13,51981.0,335.0,15.0,RT @francispouliot_: Don't worship Bitcoin Core developers. They are replaceable. I'm grateful for their work but it's often overrated. Wor…,don t worship bitcoin core developers they are replaceable i m grateful for their work but it s often overrated wor,"[worship, bitcoin, core, developer, replaceable, grateful, work, overrate, wor]",worship bitcoin core developer replaceable grateful work overrate wor,"{'neg': 0.0, 'neu': 0.574, 'pos': 0.426, 'compound': 0.6369}",0.6369,positive
2021-07-26 22:25:13,3487.0,19.0,0.0,"@pjmeersman @Stacks Case in point, let's see what Dorsey does. Will they choose to build on Stacks or create a new platform for Bitcoin defi. The price of Stacks is mostly irrelevant in this decision, me thinks.",case in point let s see what dorsey does will they choose to build on stacks or create a new platform for bitcoin defi the price of stacks is mostly irrelevant in this decision me thinks,"[case, point, let, dorsey, choose, build, stack, create, new, platform, bitcoin, defi, price, stack, irrelevant, decision, think]",case point let dorsey choose build stack create new platform bitcoin defi price stack irrelevant decision think,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'compound': 0.2732}",0.2732,positive
2021-07-26 22:25:12,2855.0,1713.0,1.0,RT @Goodnessy2: There's often a large difference between what you want and what you think you want. \n#investing #btc #Bitcoin #invest #fore…,there s often a large difference between what you want and what you think you want investing btc bitcoin invest fore,"[large, difference, want, think, want, invest, btc, bitcoin, invest, fore]",large difference want think want invest btc bitcoin invest fore,"{'neg': 0.0, 'neu': 0.755, 'pos': 0.245, 'compound': 0.1531}",0.1531,positive
2021-07-26 22:25:11,2533.0,66.0,0.0,#bitcoin\n\nAsia starting to wake up and ape into #BabyLink @BSCBabyLink \n\n#bscgem #bsc #shiba #everrise #babycake #babybake #doge #bnb,bitcoin asia starting to wake up and ape into babylink bscgem bsc shiba everrise babycake babybake doge bnb,"[bitcoin, asia, start, wake, ape, babylink, bscgem, bsc, shiba, everrise, babycake, babybake, doge, bnb]",bitcoin asia start wake ape babylink bscgem bsc shiba everrise babycake babybake doge bnb,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-26 22:25:11,1081.0,11040.0,39.0,RT @0xEarthDefi: @tourws Check our Earth Defi!\nHosted on the State-of-the-art #PolygonNetwork \n#yieldfarming and swap router\n\n👉👉 https://t.…,check our earth defi hosted on the state of the art polygonnetwork yieldfarming and swap router,"[check, earth, defi, host, state, art, polygonnetwork, yieldfarming, swap, router]",check earth defi host state art polygonnetwork yieldfarming swap router,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-26 22:25:11,1525.0,30.0,0.0,"Hey #Hexicans whats your rankings on the #PulseChain Leaderboard? I got 17,151 lol. Wednesday I might be able to throw more at it and break 10,000 position. #HEX $HEX $PLS #PLS #Bitcoin #ETH",hey hexicans whats your rankings on the pulsechain leaderboard i got 17 151 lol we

## Analysis

In [13]:
# View sentiment totals
crypto_data['vader_clean_sentiment'].value_counts()

positive    87
neutral     63
negative    12
Name: vader_clean_sentiment, dtype: int64

In [14]:
# Function for determining avg sentiment for each score in the model and overall average sentiment
def pos_neg_neutral_avg(df):
    
    positive = []
    neutral = []
    negative = []
    
    for values in df:
        if values > 0:
            positive.append(values)
        
        elif values < 0:
            negative.append(values)
        
        else:
            neutral.append(values)
    
    print(f'Positive score average for {df.name} = {round(np.mean(positive), 2)}')
    print(f'Neutral score average for {df.name} = {round(np.mean(neutral), 2)}')
    print(f'Negative score average for {df.name} = {round(np.mean(negative), 2)}')
    
    print(f'Overall crypto sentiment score is = {round(np.mean(df), 2)}')

In [15]:
# Average scores for each sentiment category, and overall sentiment score
vader_values = crypto_data.loc[:, 'vader_clean_polarity']
pos_neg_neutral_avg(vader_values)

Positive score average for vader_clean_polarity = 0.48
Neutral score average for vader_clean_polarity = 0.0
Negative score average for vader_clean_polarity = -0.38
Overall crypto sentiment score is = 0.23
